In [1]:
import sys 
sys.path.append('..')
from ml.inklabel_dataset import InkLabelDataset
import yaml
import lightning as pl
from denoising_diffusion_pytorch import Unet, GaussianDiffusion
import torch
segment_ids = yaml.safe_load(open('../configs/segment_ids.yaml', 'r'))
print(segment_ids['segment_ids'])

SAMPLE_SIZE = 512

{'segments': ['20231210121321', '20231106155350', '20231005123336', '20230820203112', '20230620230619', '20230826170124', '20230702185753', '20230522215721', '20230531193658', '20230520175435', '20230903193206', '20230902141231', '20231007101615', '20230929220924', 'recto', 'verso', '20231016151000', '20231012184423', '20231031143850']}


/home/npopkov/vesuvius_gcml/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ink_label_dataset = InkLabelDataset(segment_ids['segment_ids']['segments'],SAMPLE_SIZE)
ink_label_dataloader = torch.utils.data.DataLoader(ink_label_dataset, batch_size=64, shuffle=True, num_workers=25)

Creating InkLabelDataset with 19 segments...
Error processing segment recto: invalid literal for int() with base 10: 'recto'Error processing segment verso: invalid literal for int() with base 10: 'verso'

An error occurred while initializing the Volume class: URL not found for scroll: None, energy: None, resolution: None, segment: 20231012184423An error occurred while initializing the Volume class: URL not found for scroll: None, energy: None, resolution: None, segment: 20231031143850
Load the canonical scroll 1 with Volume(type="scroll", scroll_id=1, energy=54, resolution=7.91)

Load the canonical scroll 1 with Volume(type="scroll", scroll_id=1, energy=54, resolution=7.91)If loading another part of the same physical scroll use for instance Volume(type="scroll", scroll_id="1b", energy=54, resolution=7.91)

If loading another part of the same physical scroll use for instance Volume(type="scroll", scroll_id="1b", energy=54, resolution=7.91)Load a segment (e.g. 20230827161847) with Volume

/home/npopkov/vesuvius_gcml/notebooks/../ml/inklabel_dataset.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  self.samples = [torch.tensor(sample, dtype=torch.float32) for sample in self.samples]


Created 3
Created 1470 samples from ink labels.


In [3]:
model = Unet(
    dim = 32,
    channels= 1,
    dim_mults = (1, 2, 4),
    flash_attn = False
)

diffusion = GaussianDiffusion(
    model,
    image_size = SAMPLE_SIZE,
    
    timesteps = 1000  
)

class LightningWrapper(pl.LightningModule):
    def __init__(self, diffusion):
        super().__init__()
        self.diffusion = diffusion

    def forward(self, x):
        if len(x.shape) == 3:
            x = x.unsqueeze(1)
        
        return self.diffusion(x)

    def training_step(self, batch, batch_idx):
        x = batch
        pred = self(x)
        loss = torch.nn.functional.mse_loss(pred, x)
        self.log('train_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.diffusion.parameters(), lr=3e-4)
        return optimizer
    
lightning_model = LightningWrapper(diffusion)
trainer = pl.Trainer(max_epochs=1000, accelerator="gpu", devices=1, callbacks=[pl.pytorch.callbacks.early_stopping.EarlyStopping(monitor="train_loss", patience=15, mode="min")])

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/npopkov/vesuvius_gcml/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [4]:
trainer.fit(lightning_model, ink_label_dataloader)
torch.save(lightning_model.state_dict(), 'ink_label_diffusion.pth')
print("Training complete. Model saved as 'ink_label_diffusion.pth'.")

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name      | Type              | Params | Mode 
--------------------------------------------------------
0 | diffusion | GaussianDiffusion | 2.6 M  | train
--------------------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.431    Total estimated model params size (MB)
298       Modules in train mode
0         Modules in eval mode
/home/npopkov/vesuvius_gcml/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] 

OutOfMemoryError: CUDA out of memory. Tried to allocate 44.01 GiB. GPU 0 has a total capacity of 23.65 GiB of which 1.69 GiB is free. Including non-PyTorch memory, this process has 21.94 GiB memory in use. Of the allocated memory 21.34 GiB is allocated by PyTorch, and 157.42 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)